原理剖析：

定义时：`@timer` 语法糖在解释阶段就执行了 `heavy_task = timer(heavy_task)`。此时，`timer` 函数被调用，其参数 `func` 就是原 `heavy_task` 函数对象。 

创建闭包：`timer` 内部定义了 `wrapper` 函数，并且 `wrapper` 引用了外部变量 `func`。由于 `wrapper` 引用了 `func`，一个闭包就形成了，
`func` 被“冻结”在 `wrapper` 的环境中。  

返回闭包：`timer` 返回的是 `wrapper` 函数对象本身，而不是它的调用结果。这个被返回的 `wrapper` 函数，就是新的 `heavy_task`。  

调用时：当我们执行 `heavy_task(10000)`，实际上执行的是 `wrapper(10000)`。这个 `wrapper` 可以访问它“记住”的 `func`（即原函数），  
从而实现了在调用原函数前后添加计时逻辑的能力。

这就是装饰器最精妙的地方：它在不侵入原函数代码的前提下，通过闭包将新功能“包裹”在原函数之外

In [2]:
import time
from functools import wraps  # 一个用于保留原函数元信息的工具

def timer(func):
    """装饰器：记录函数执行时间"""
    @wraps(func)  # 重要：将原函数的__name__等属性复制到闭包函数
    def wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)  # 执行被装饰的原函数
        end_time = time.perf_counter()
        print(f"[timer] 函数 `{func.__name__}` 耗时 {end_time - start_time:.4f} 秒")
        return result
    return wrapper  # 返回闭包函数

# 使用装饰器
@timer  # 等价于 `heavy_task = timer(heavy_task)`
def heavy_task(n):
    """模拟一个耗时计算"""
    sum = 0
    for i in range(n):
        sum += i * i
    return sum

if __name__ == '__main__':
    # 调用
    # 输出：
    # [timer] 函数 `heavy_task` 耗时 0.0008 秒
    # 计算结果：333283335000
    result = heavy_task(10000)
    print(f"计算结果：{result}")


[timer] 函数 `heavy_task` 耗时 0.0008 秒
计算结果：333283335000


惰性求值与延迟计算  
核心思想：闭包可以用于封装一个尚未执行的计算过程。只有当真正需要结果时，才触发计算，并且计算结果可以被缓存起来，避免重复计算。这是一种简单的记忆化实现。  
经典代码：惰性属性（使用闭包模拟property）  
原理剖析：

延迟执行：`@lazy_property` 装饰器将属性访问器 `expensive_computation` 替换为一个闭包 `_lazy_property`。  
缓存逻辑：这个闭包函数检查实例中是否存在一个特定名称（如 `_lazy_expensive_computation`）的缓存属性。如果没有，就调用原函数进行计算，并将结果存入缓存。  
闭包的作用：装饰器工厂 `lazy_property` 中的闭包，记住了原函数 `fn` 和生成的缓存属性名 `attr_name`。这使得每个被装饰的属性都有自己独立的缓存键，逻辑完全隔离。  
优势：将“计算-缓存”这一通用逻辑从业务代码中抽离出来，任何被 `@lazy_property` 装饰的方法都会自动获得惰性求值的能力。  
这正是 “面向切面编程” 思想的体现，闭包是实现它的轻量级工具。

In [3]:
class ExpensiveObject:
    def __init__(self, data_source):
        self.data_source = data_source
        self._cached_data = None  # 用于缓存计算结果

    @property
    def expensive_computation(self):
        """一个计算成本很高的属性，我们希望它只算一次"""
        if self._cached_data is None:
            print("正在进行昂贵的计算...")
            # 模拟耗时操作
            result = 0
            for i in range(self.data_source):
                result += i
            self._cached_data = result  # 缓存结果
        return self._cached_data

# 但使用闭包，我们可以将“惰性初始化”的逻辑封装得更通用
def lazy_property(fn):
    """一个实现惰性属性的装饰器（高级闭包应用）"""
    attr_name = '_lazy_' + fn.__name__
    @property
    @wraps(fn)
    def _lazy_property(self):
        if not hasattr(self, attr_name):
            setattr(self, attr_name, fn(self))  # 第一次访问时计算并缓存
        return getattr(self, attr_name)
    return _lazy_property

class BetterExpensiveObject:
    def __init__(self, data_source):
        self.data_source = data_source

    @lazy_property
    def expensive_computation(self):
        print("Better: 正在进行昂贵的计算...")
        result = 0
        for i in range(self.data_source):
            result += i
        return result

# 使用示例
print("=== 使用闭包实现的惰性属性 ===")
obj = BetterExpensiveObject(1000000)
print("第一次访问属性:")
val1 = obj.expensive_computation  # 触发计算
print(f"结果: {val1}")
print("第二次访问属性:")
val2 = obj.expensive_computation  # 直接返回缓存值，无打印输出
print(f"结果: {val2}, 它是从缓存读取的。")

=== 使用闭包实现的惰性属性 ===
第一次访问属性:
Better: 正在进行昂贵的计算...
结果: 499999500000
第二次访问属性:
结果: 499999500000, 它是从缓存读取的。
